# Проект 1. Предобработка данных

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### План проекта
1. Изучение общей информации
2. Преобработка данных  
   2.1 Обработка признаков    
   2.2 Замена типа данных  
   2.3 Обработка дубликатов  
   2.4 Лемматизация  
   2.5 Категоризация данных  
3. Анализ факторов, влияющих на возврат кредита в срок  
   3.1 Зависимость между наличием детей и возвратом кредита в срок  
   3.2 Зависимость между семейным положением и возвратом кредита в срок  
   3.3 Зависимость между уровнем дохода  и возвратом кредита в срок  
   3.4 Влияние различных целей кредита на его возврат в срок   
4. Общий вывод  

### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


### Вывод: 
В таблице 12 столбцов и 21525 строк. Количество значений в столбцах различается, что свидетельствует о наличии пропущенных значений, в частности в столбцах 'days_employed' и 'total_income'. В данных присутствует 7 столбцов с числовыми данными и 5 столбцов с объектами.

### Шаг 2. Предобработка данных

### 2.1 Обработка пропусков

In [2]:
# рассчитаем количество пропущенных значений в столбце 'days_employed'
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [3]:
#рассчитаем количество пропущенных значений в столбце с доходом 'total_income'
print(data[data['total_income'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [4]:
#выявим артфефакты - кол-во отрицательных значений в столбцах таблицы
data[data['days_employed'] < 0]['days_employed'].count()

15906

In [5]:
data[data['total_income'] < 0]['total_income'].count()

0

In [6]:
data[data['children'] < 0]['children'].count()

47

In [7]:
#преобразуема отрицательные значения трудового стажа и кол-во детей в положительные значения
data[['days_employed', 'children']] = data[['days_employed', 'children']].abs()

In [8]:
#проверим наличие аномальных значений в столбце с количеством дней занятости 
data[data['days_employed'] > (data['dob_years'] - 16)*365]['days_employed'].count()

3608

In [9]:
#предположим, что в выявленных строках, где дни занятости принимают аномально высокие значения, значения выражены не в днях, а в часах. 
#для получения искомого значения, преобразуем данные строки в дни
selected_rows = data[data['days_employed'] > (data['dob_years'] - 16)*365]
transformed_values_for_employment = selected_rows ['days_employed']/24

In [10]:
transformed_values_for_employment

4        14177.753002
18       16678.380705
24       14106.331371
25       15147.853723
30       13982.569521
             ...     
21505    14121.036100
21508    16104.071420
21509    15090.043922
21518    15583.154618
21521    14330.725172
Name: days_employed, Length: 3608, dtype: float64

In [11]:
#подставим скорректированные аномальные значения в искомый датасет
data.loc[data['days_employed'] > (data['dob_years'] - 16)*365, 'days_employed'] = transformed_values_for_employment

In [12]:
#Предположим, что величина трудового стажа зависит от возраста заемщика. 
#Определим диапазон возраста заемщиков. 
data['dob_years'].min()

0

In [13]:
data['dob_years'].max()

75

In [14]:
#учитвая, что диапазон возраста заемщиков от 0 до 75,классифицируем их на возрастные категории для дальнейшего расчета медианы для каждой из них
def age_group(age):
    if age < 30:
        return 'молодые'
    if 31 <= age <= 60:
        return 'зрелые'
    return 'пенсионеры'

In [15]:
#создадаим стоблец с возрастными категориями в искомом датасете
data['age_group'] = data['dob_years'].apply(age_group)

In [16]:
#рассчитаем медианные значения трудового стажа в зависимости от ворзастной категории, к которой принадлежит заемщик
data.groupby('age_group')['days_employed'].median()

age_group
зрелые         2312.623375
молодые         963.613672
пенсионеры    14359.556849
Name: days_employed, dtype: float64

In [17]:
#создадим словарь, состоящий из медианных значений стажа и соответствующим им возрастных категорий
dict = {'зрелые':2312.623375, 'молодые': 963.613672, 'пенсионеры': 14359.556849}

In [18]:
#создадим функцию для заполнения пропуском значениями из словаря dict
def fill_missing_days(row):
    if pd.isna(row['days_employed']):
        row['days_employed'] == dict[row['age_group']]
        return dict[row['age_group']]
    else:
        return row['days_employed']

In [19]:
#заполним пропуски в столбце 'days_employed' с помощью созданной функции
data['days_employed'] = data.apply(fill_missing_days, axis=1)

In [20]:
#предположим, что ежемесячный доход заемщика зависит от типа его занятости
#сгруппируем датасет по типу занятости и рассчитает медианное значение дохода для каждой категории
new_data = data.groupby('income_type')['total_income'].median()
print(new_data)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [21]:
#создадим словарь из типа занятости и соответствуюшего ему дохода
dict_2 = {'безработный':  131339.751676, 'в декрете': 53829.130729, 'госслужащий': 150447.935283, 'компаньон': 172357.950966, 'пенсионер': 118514.486412, 'предприниматель': 499163.144947, 'сотрудник' :  142594.396847, 'студент' : 98201.625314}

In [22]:
#создадим функцию для заполнения пропуском значениями из словаря dict_2
def fill_missing_values(row):
    if pd.isna(row['total_income']):
        row['total_income'] == dict_2[row['income_type']]
        return dict_2[row['income_type']]
    else:
        return row['total_income']

In [23]:
#заполним пропуски в столбце 'total_income' с помощью созданной функции
data['total_income'] = data.apply(fill_missing_values, axis = 1)

In [24]:
#проверим заполнились ли пропуски в столбцах 'total_income' и 'days_employed'
print(data[data['total_income'].isnull()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64


In [25]:
print(data[data['days_employed'].isnull()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64


### ВЫВОД
В результате предварительного анализа были выявлены пропуски в столбцах с трудовым стажем, выраженным в днях ('days_employed'), и ежемесячным доходом ('total_income'). В целях избежать смещения в выборке пропуски в трудовом стаже заполнялись медианными значениями в зависимости от возрастной группы, к которой принадлежит заемщик, а пропуски в величине дохода - медианными значениями в зависимости от типа занятости, характерного для заемщика. Пропуски в данных могли быть результатом технической ошибки, например, при переносе информации из письменной анкеты в электронную. Также в данных были выявлены артефакты: н-р, отрицательные значения в стаже, который могли быть результатом вычитания переменных в неверном порядке (дата начала работы - дата завершения работы).

### 2.2 Замена типа данных

In [26]:
data['days_employed'].value_counts()

2312.623375     1584
963.613672       309
14359.556849     281
5585.832554        1
14583.152504       1
                ... 
262.922745         1
914.167360         1
339.153770         1
2849.351119        1
582.538413         1
Name: days_employed, Length: 19354, dtype: int64

In [27]:
#Приведем значения в столбце с количеством дней занятости к целочисленному виду
data['days_employed'].astype(int)

0         8437
1         4024
2         5623
3         4124
4        14177
         ...  
21520     4529
21521    14330
21522     2113
21523     3112
21524     1984
Name: days_employed, Length: 21525, dtype: int64

In [28]:
#Приведем значения в столбце в доходом к целочисленному виду
data['total_income'].astype(int)

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

In [29]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,зрелые
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,зрелые
2,0.0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,зрелые
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,зрелые
4,0.0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,зрелые


### Вывод
Приобразовали значения дней занятости и дохода из вещественного типа к целочисленному с помощью функции astype(). Функция to_numeric() не подходит в данном случае, поскольку используется для преобразования значений типа 'string'.


### 2.3 Обработка дубликатов

In [30]:
#Посмотрим на уникальные значения в разных столбцах таблицы
data['gender'].value_counts()
data['income_type'].value_counts()
data['purpose'].value_counts()
data['family_status'].value_counts()
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [31]:
#т.к. значения в столбце 'education' отличаются регистром, приведем их к единому нижнему регистру
data['education'] = data['education'].str.lower()

In [32]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [33]:
#рассчитаем число полных дубликатов
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 71


In [34]:
#посмотрим на строки с дубликатами
data[data[['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'age_group']].duplicated() == True].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
2849,0.0,2312.623375,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,покупка жилья для семьи,зрелые
3290,0.0,2312.623375,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу,зрелые
4182,1.0,2312.623375,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба,зрелые
4851,0.0,2312.623375,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба,зрелые
5557,0.0,2312.623375,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу,зрелые
6312,0.0,14359.556849,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,строительство жилой недвижимости,пенсионеры
7808,0.0,2312.623375,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы,зрелые
7921,0.0,14359.556849,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы,пенсионеры
7938,0.0,14359.556849,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы,пенсионеры
8583,0.0,2312.623375,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514.486412,дополнительное образование,зрелые


In [35]:
#удалим строки с полными дубликатами
data = data.drop_duplicates()

In [36]:
#рассчитаем количество дубликатов в таблице
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 0


### Вывод
Для выявления строк-полных дубликатов в таблице предварительно значения в столбцах были приведены к единому регистру. В результате был выявлен и удален 71 дубликат. Дубликаты могли появиться в результате повторной ошибочной регистрации данных одних и тех же клиентов, но в разных регистрах. 

<div class="alert alert-success">

### Комментарий ревьюера

Безупречная работа! Здорово, что перед удалением дублей проверила регистры

</div>

### 2.4 Лемматизация

In [37]:
#посмотрим на уникальные значения в столбце с целями кредита
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [38]:
#установим пакет для выделения лемм в строках
from pymystem3 import Mystem
m = Mystem()
data['lemmas'] = data['purpose'].apply(m.lemmatize)

In [39]:
data['lemmas']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: lemmas, Length: 21454, dtype: object

In [40]:
#объединим леммы в строки
data['lemmas'] = data['lemmas'].str.join('')

In [41]:
data['lemmas']

0                     покупка жилье\n
1           приобретение автомобиль\n
2                     покупка жилье\n
3        дополнительный образование\n
4                   сыграть свадьба\n
                     ...             
21520              операция с жилье\n
21521           сделка с автомобиль\n
21522                  недвижимость\n
21523    на покупка свой автомобиль\n
21524         на покупка автомобиль\n
Name: lemmas, Length: 21454, dtype: object

In [42]:
#удалим из строк лемму '\n'
data['lemmas'] = data['lemmas'].str.rstrip('\n')

In [43]:
data['lemmas'].unique()

array(['покупка жилье', 'приобретение автомобиль',
       'дополнительный образование', 'сыграть свадьба',
       'операция с жилье', 'образование', 'на проведение свадьба',
       'покупка жилье для семья', 'покупка недвижимость',
       'покупка коммерческий недвижимость', 'покупка жилой недвижимость',
       'строительство собственный недвижимость', 'недвижимость',
       'строительство недвижимость', 'на покупка подержать автомобиль',
       'на покупка свой автомобиль',
       'операция с коммерческий недвижимость',
       'строительство жилой недвижимость', 'жилье',
       'операция со свой недвижимость', 'автомобиль',
       'заниматься образование', 'сделка с подержанный автомобиль',
       'получение образование', 'свадьба',
       'получение дополнительный образование', 'покупка свой жилье',
       'операция с недвижимость', 'получение высокий образование',
       'свой автомобиль', 'сделка с автомобиль', 'профильный образование',
       'высокий образование', 'покупка жилье 

### Вывод
В результате лемматизации мы получили леммы, на основании анализа которых можно выделить следующие четыре категории целей кредитования: недвижимость, автомобиль, образование, свадьба. С соответствии с полученными леммами можно произвести дальнейшую категоризацию целей кредитования. 

### 2.5 Категоризация данных

In [44]:
#определим функцию, преобразующую лемматизированную строку с целью кредита в одну из установленных четырех категорий 
def purpose_categories(column):
    if 'образование' in column:
        return 'образование'
    if 'автомобиль' in column:
        return 'автомобиль'
    if 'свадьба' in column: 
        return 'свадьба'
    if 'недвижимость' or 'жилье' in column:
        return 'недвижимость'
    

    return 'неизвестная категория'

In [45]:
#применим функцию к столбцу с целью кредитования
data['purpose_category'] = data['lemmas'].apply(purpose_categories)

In [46]:
#узнаем, сколько кредитов выдано на каждую цель
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

In [47]:
#категоризуем данные в зависимости от величины ежемесячного дохода
#определим максимальное и минимальное значение дохода
data['total_income'].max()

2265604.028722744

In [48]:
data['total_income'].min()

20667.26379327158

In [49]:
#определим функцию, выделяющую категории дохода
def income_group(value):
    if value < 40000:
        return 'низкий уровень'
    if 40000 <= value <= 75000: 
        return 'ниже среднего уровня'
    if 75000 < value <= 110000:
        return 'средний уровень'
    if 110000 < value <= 145000: 
        return 'выше среднего уровня'
    if 145000 < value <= 180000: 
        return 'высокий уровень'
    
    return 'высокий уровень +'

In [50]:
#применим функцию к столбцу с уровнем дохода
data['income_category'] = data['total_income'].apply(income_group)

In [51]:
data['income_category'].value_counts()

высокий уровень +       6423
выше среднего уровня    5488
высокий уровень         3901
средний уровень         3777
ниже среднего уровня    1740
низкий уровень           125
Name: income_category, dtype: int64

In [52]:
#Посмотрим на структуру выборки с точки зрения доли каждой категории в общей выборке
print((data['age_group'].value_counts())/(data['age_group'].value_counts().sum()))

зрелые        0.722942
молодые       0.152932
пенсионеры    0.124126
Name: age_group, dtype: float64


In [53]:
print((data['income_category'].value_counts())/(data['income_category'].value_counts().sum()))

высокий уровень +       0.299385
выше среднего уровня    0.255803
высокий уровень         0.181831
средний уровень         0.176051
ниже среднего уровня    0.081104
низкий уровень          0.005826
Name: income_category, dtype: float64


In [54]:
print((data['purpose_category'].value_counts())/(data['purpose_category'].value_counts().sum()))

недвижимость    0.503915
автомобиль      0.200708
образование     0.187051
свадьба         0.108325
Name: purpose_category, dtype: float64


In [55]:
print((data['children'].value_counts())/(data['children'].value_counts().sum()))

0.0     0.656801
1.0     0.226298
2.0     0.095646
3.0     0.015382
20.0    0.003542
4.0     0.001911
5.0     0.000420
Name: children, dtype: float64


In [56]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.544327,4498.877195,43.271231,0.817097,0.973898,0.081150,1.653201e+05
std,1.381726,5249.840002,12.570822,0.548674,1.421567,0.273072,9.818730e+04
min,0.000000,4.540293,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,963.613672,33.000000,1.000000,0.000000,0.000000,1.076239e+05
50%,0.000000,2312.623375,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5039.006631,53.000000,1.000000,1.000000,0.000000,1.958209e+05
max,20.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод
Были выделены четыре категории с целями кредитования и четыре категории доходов заемщиков для упрощения дальнейшего анализа зависимости между уровнем дохода, целями кредита и доли заемщиков, имеющих задолженность.

### Шаг 3. Анализ факторов, влияющих на возврат кредита в срок

### 3.1 Зависимость между наличием детей и возвратом кредита в срок

In [57]:
data_children_pivot = data.pivot_table(index=['debt'], columns = 'children', values = 'gender', aggfunc = 'count')

In [58]:
#рассчитаем долю заемщиков с задолженностью для каждой категории родителей
fraction_series = data.groupby('children')['debt'].mean()
data_children_pivot = data_children_pivot.append(fraction_series, ignore_index = True)
data_children_pivot = data_children_pivot.rename(index={0: 'количество заемщиков без задолженности', 1: 'количество должников', 2 : 'доля должников'})

In [59]:
#рассчитаем % просрочки для всей таблицы
data_children_pivot['все категории'] = data_children_pivot.loc['количество заемщиков без задолженности':'количество должников', 0:7].sum(axis=1)

In [60]:
data_children_pivot['все категории'] = data_children_pivot['все категории'].fillna(data_children_pivot['все категории'][1]/(data_children_pivot['все категории'][0] + data_children_pivot['все категории'][1]))

In [61]:
data_children_pivot.loc['доля должников'] = data_children_pivot.loc['доля должников'].apply('{:.2%}'.format)

In [62]:
display(data_children_pivot)

children,0.0,1.0,2.0,3.0,4.0,5.0,20.0,все категории
количество заемщиков без задолженности,13028,4410,1858,303,37,9,68,19645
количество должников,1063,445,194,27,4,NaN,8,1733
доля должников,7.54%,9.17%,9.45%,8.18%,9.76%,0.00%,10.53%,8.11%


### Вывод
В данных прослеживается нелинейная тенденция: по мере увеличения количества детей с 0 до 2 и с 4 до 20 доля просроченной задолженности увеличивается, однако в семьях с 3 детьми она ниже, чем в семьях с одним ребенком и двумя детьми. При этом в семьях, где дети отсутствуют, доля просроченной задолженности на 57 б.п. ниже, чем в среднем для всех категорий семей. Полученная динамика просроченной задолженности может быть обусловлена тем, что с увеличением количества детей возрастают потребительские расходы семей относительно их доходов, что приводит к их более низкой платежеспособности, чем в семьях без детей. Более высокая платежеспособность детей с тремя детьми, чем с двумя детьми или одним ребенком, может быть обусловлена, например, более высокими государственными пособиями на третьего ребенка, что повышает совокупный доход семей.</font>

### 3.2 Зависимость между семейным положением и возвратом кредита в срок

In [57]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [58]:
#для сбалансированности выборки объединим категории не замужем/не женат, в разводе, вдовец/вдова в одну категории 'не состоит в браке'
def family_status_cat(value):
    if value == 'Не женат / не замужем' or value == 'в разводе' or value == 'вдовец / вдова':
        return 'не состоит в браке'
    if value == 'женат / замужем':
        return 'женат / замужем'
    if value == 'гражданский брак':
        return 'гражданский брак'

In [59]:
data['new_family_status_id'] = data['family_status'].apply(family_status_cat)

In [60]:
data['new_family_status_id'].value_counts()

женат / замужем       12339
не состоит в браке     4964
гражданский брак       4151
Name: new_family_status_id, dtype: int64

In [61]:
#посмотрим на структуру данных
print((data['new_family_status_id'].value_counts())/(data['new_family_status_id'].value_counts().sum()))

женат / замужем       0.575138
не состоит в браке    0.231379
гражданский брак      0.193484
Name: new_family_status_id, dtype: float64


In [62]:
data_family_status_pivot = data.pivot_table(index=['debt'], columns = 'new_family_status_id', values = 'gender', aggfunc = 'count')

In [63]:
#рассчитаем долю должников в каждой категории 
fraction_family_status = (data_family_status_pivot.loc[1, :]/(data_family_status_pivot.loc[0, :] + data_family_status_pivot.loc[1, :]))
data_family_status_pivot = data_family_status_pivot.append(fraction_family_status, ignore_index = True)
data_family_status_pivot = data_family_status_pivot.rename(index={0: 'количество заемщиков без задолженности', 1: 'количество должников', 2 : 'доля должников'})

In [64]:
#рассчитаем % просрочки для всей таблицы и добавим соответствующий столбец
data_family_status_pivot['все категории'] = data_family_status_pivot.loc['количество замещиков без задолженности':'количество должников', 'гражданский брак':'не состоит в браке'].sum(axis=1)

In [65]:
data_family_status_pivot['все категории'] = data_family_status_pivot['все категории'].fillna(data_family_status_pivot['все категории'][1]/(data_family_status_pivot['все категории'][0] + data_family_status_pivot['все категории'][1]))

In [66]:
data_family_status_pivot.loc['доля должников'] = data_family_status_pivot.loc['доля должников'].apply('{:.2%}'.format)

In [67]:
display(data_family_status_pivot)

new_family_status_id,гражданский брак,женат / замужем,не состоит в браке,все категории
количество заемщиков без задолженности,3763,11408,4542,19713
количество должников,388,931,422,1741
доля должников,9.35%,7.55%,8.50%,8.12%


### Вывод
На основании полученной сводной таблицы можно выдвинуть гипотезу о том, что законный брак оказывает положительное влияние на добросовестность заемщиков. Среди лиц, состоящих в законодательно закрепленном браке, доля должников ниже на 57 б.п., чем в среднем для всех категорий лиц, и на 95 б.п. ниже, чем для лиц, не состоящих в браке. При этом для лиц, состоящих в гражданском браке ситуация обратная - доля лиц с просроченной задолженностью среди них на 123 б.п. выше, чем в среднем для всех категорий заемщиков, и на 85 б.п. выше, чем для лиц, не состоящих в браке. Наблюдаемый результат может быть связан с тем, что у пар в законодательно закрепленном браке формируется общий бюджет, в результате чего падение доходов одной из сторон союза может компенсироваться стабильным доходом другой стороны, что увеличивает платежеспособность заемщика. </font>

### 3.3 Зависимость между уровнем дохода и возвратом кредита в срок

In [68]:
income_category_pivot = data.pivot_table(index = ['debt'], columns = 'income_category', values = 'gender', aggfunc = 'count')
income_category_pivot = income_category_pivot.append(income_category_pivot.loc[1, :]/(income_category_pivot.loc[0, :] + income_category_pivot.loc[1, :]), ignore_index = True)
income_category_pivot = income_category_pivot.rename(index={0: 'количество заемщиков без задолженности', 1: 'количество должников', 2 : 'доля должников'})
#рассчитаем % просрочки для всей таблицы и добавим соответствующий столбец
income_category_pivot['все категории'] = income_category_pivot.loc['количество замещиков без задолженности':'количество должников', 'высокий уровень':'средний уровень'].sum(axis=1)
income_category_pivot['все категории'] = income_category_pivot['все категории'].fillna(income_category_pivot['все категории'][1]/(income_category_pivot['все категории'][0] + income_category_pivot['все категории'][1]))
income_category_pivot.loc['доля должников'] = income_category_pivot.loc['доля должников'].apply('{:.2%}'.format)

In [69]:
display(income_category_pivot)

income_category,высокий уровень,высокий уровень +,выше среднего уровня,ниже среднего уровня,низкий уровень,средний уровень,все категории
количество заемщиков без задолженности,3575,5945,5006,1614,115,3458,19713
количество должников,326,478,482,126,10,319,1741
доля должников,8.36%,7.44%,8.78%,7.24%,8.00%,8.45%,8.12%


### Вывод
На основании сводной таблицы можно заключить, что взаимосвязь между уровнем дохода и долей просроченной задолженности нелинейная. Наиболее высокая доля просроченной задолженности наблюдается у заемщиков с доходами выше среднего уровня (т.е. с доходами в диапазоне от 110 тыс. до 145 тыс. руб.), за ней следуют заемщики с доходами на среднем уровне (% просрочки на 33 б.п. выше, чем в среднем для всех категорий заемщиков) и заемщики с высоким уровнем дохода (% просрочки на 24 б.п. выше, чем в среднем для всех категорий заемщиков).Наиболее низкий уровень задолженности наблюдается у заемщиков с доходами ниже среднего уровня (% просрочки на 88 б.п. ниже,  чем в среднем для всех категорий) и самой высокодоходной группы (ежемесячный доход > 180 тыс. руб - % просрочки на 68 б.п. ниже, чем в среднем для всех категорий). Таким образом, получен на первый взгляд контруинтуитивный результат - лица с низким уровнем дохода реже имеют задолженность по кредитам, чем заемщики с высоким уровнем дохода и доходом выше среднего. Полученный результат может быть обусловлен, в частности, меньшей величиной займа, выдаваемого клиентам с низким уровнем дохода или более значимой роли их кредитной истории при одобрении им кредитов.</font>

### 3.4 Влияние различных целей кредита на его возврат в срок 

In [70]:
purpose_category_pivot = data.pivot_table(index = ['debt'], columns = 'purpose_category', values = 'gender', aggfunc = 'count')
purpose_category_pivot = purpose_category_pivot.append(purpose_category_pivot.loc[1, :]/(purpose_category_pivot.loc[0, :] + purpose_category_pivot.loc[1, :]), ignore_index = True)
purpose_category_pivot = purpose_category_pivot.rename(index={0: 'количество заемщиков без задолженности', 1: 'количество должников', 2 : 'доля должников'})
#рассчитаем % просрочки для всей таблицы и добавим соответствующий столбец
purpose_category_pivot['все категории'] = purpose_category_pivot.loc['количество замещиков без задолженности':'количество должников', 'автомобиль':'свадьба'].sum(axis=1)
purpose_category_pivot['все категории'] = purpose_category_pivot['все категории'].fillna(purpose_category_pivot['все категории'][1]/(purpose_category_pivot['все категории'][0] + purpose_category_pivot['все категории'][1]))
purpose_category_pivot.loc['доля должников'] = purpose_category_pivot.loc['доля должников'].apply('{:.2%}'.format)

In [71]:
display(purpose_category_pivot)

purpose_category,автомобиль,недвижимость,образование,свадьба,все категории
количество заемщиков без задолженности,3903,10029,3643,2138,19713
количество должников,403,782,370,186,1741
доля должников,9.36%,7.23%,9.22%,8.00%,8.12%


### Вывод
Наибольший процент должников наблюдается в сегменте кредитования с целью покупки автомобиля и получения образования. Заемщики, выплачивающие кредит, одобренный в целях приобретения недвижимости, наиболее добросовестные - доля просроченной задолженности в данной группе самая низкая (на 89 б.п. ниже, чем в среднем для всех групп заемщиков). Возможная интерпретация: в случае приобретения жилья заемщики проходят более строгую проверку с точки зрения их текущей платежеспособности в силу более значительной суммы кредита и более длительного срока кредитования, чем в сиутации получения кредита на приобретение автомобиля или получение образования. Кроме того, решение о приобретении жилья предположительно более тщательно и заблаговременно планируется заемщиком с точки зрения имеющихся резервов и будущих денежных поступлений, чем решение о займах на иные цели.

### Шаг 4. Общий вывод
Цель представленного проекта заключалась в выявлении влияния семейного положения и количества детей на факт погашения кредита в срок.  Для обнаружения данной зависимости в анализе использовались данные о платежеспособности клиентов банка, включающие информацию о количестве детей в семье, трудовом стаже, возрасте клиентов, уровне образования, семейном положении, ежемесячном доходе, целе получения кредита и факте наличия просроченной задолженности по кредиту.  

На этапе предобработки данных были выявлены пропущенные значения в стобцах с трудовым стажем и ежемесячным доходом. Поскольку доля пропущенных значений существенная (10% всей выборки), удаление пропущенных значений привело бы к потере значительной информации. Для предотвращения смещения в выборке пропуски в значениях трудового стажа заполнялись медианными значениями в зависимости от возрастной группы, к которой принадлежит заемщик, а пропуски в величине дохода - медианными значениями на основании типа занятости, характерного для заемщика. 
Кроме того, в данных было выявлено и устранено два артефакта: отрицательные значения трудового стажа и аномально большие значения количества дней занятости. Первый артефакт может быть следствием ошибочного порядка вычитания переменных (дата начала работы - дата завершения работы), а второй - ошибочного измерения (расчета занятости в часах, а не днях).
В результате проведения процедуры предобработки данных, включая удаление дубликатов и лемматизацию, можно привести следующие описательные характеристики. Средний возраст заемщика в выборке - 43 года (72% клиентов находятся в зрелом возрасте - от 30 до 60 лет). Большая часть заемщиков финансово состоятельная - клиентов с ежемесячным доходом выше 110 тыс. руб. - 73,8%, с доходом ниже 75 тыс. руб. - 8,7% (среднее значение дохода в выборке - 165320 руб.). В официальном браке состоят 57,5% заемщиков, в гражданском браке - 19,3%. При этом 66% заемщиков не имеют детей, 23% - являются родителями одного ребенка. Наиболее популярная цель кредитования - приобретение недвижимости (50,4% заемщиков), за ней следуют покупка автомобиля (20% заемщиков), получение образования (18,7%) и проведение свадьбы (10,8%). Средняя доля просроченной задолженности по всей выборке составляет 8,12%.

На основании сводных таблиц была проанализирована взаимосвязь между возрастом заемщика, доходом, количеством детей, семейным положением и возвратом кредита в срок, и получены следующие выводы:
1. По мере увеличения количества детей, доля просроченной задолженности возрастает:в частности, в семьях, где отсутствуют дети, она составляет 7,54% (на 57 б.п. ниже, чем в среднем по выборке), в семьях с 4 детьми - 9,76% (на 165 б.п. выше, чем в среднем по выборке). Полученная динамика может объясняться тем, что с увеличением количества детей в семье снижается  ее платежеспособность  из-за роста потребительских расходов относительно доходов. 
2. Официальный брак оказывает положительное влияние на добросовестность заемщиков. Доля должников среди данной категории клиентов на 57 б.п. ниже, чем в среднем по выборке, и на 95 б.п. ниже, чем для лиц, не состоящих в браке. Однако гражданский брак оказывает противоположный эффект на платежеспособность заемщиков - доля просроченной задолженности в данной категории на 123 б.п. выше, чем в среднем по всей выборке. Возможная интерпретация полученного результата: у лиц в официальном браке формируется общий бюджет, в результате чего падение доходов одной из сторон союза может компенсироваться стабильным доходом другой стороны, что обеспечивает неизменную платежеспособность клиента.
3. Лица с низким уровнем дохода реже имеют задолженность по кредитам, чем заемщики с высоким уровнем дохода и доходом выше среднего. Полученный результат может быть обусловлен, в частности, меньшей величиной займа, выдаваемого клиентам с низким уровнем дохода или более значимой роли их кредитной истории при одобрении им кредитов.
4. Наибольший процент должников наблюдается в сегменте кредитования с целью покупки автомобиля и получения образования. Заемщики, выплачивающие кредит, одобренный в целях приобретения недвижимости, наиболее добросовестные - доля просроченной задолженности в данной группе самая низкая (на 89 б.п. ниже, чем в среднем для всех групп заемщиков). Возможная интерпретация: в случае приобретения жилья заемщики проходят более строгую проверку с точки зрения их текущей платежеспособности в силу более значительной суммы кредита и более длительного срока кредитования, чем в ситуации получения кредита на приобретение автомобиля или получение образования.

Таким образом, на основании предварительного анализа данных можно выдвинуть гипотезу об отрицательном влиянии количества детей и положительном влиянии официального брака на платежеспособность клиентов. Проверка данной гипотезы может осуществляться в дальнейшем в рамках логистической регрессии за счет включения соответствующих дамми-переменных в модель.
